# CEE6501 — Lecture 3.2

## The Direct Stiffness Method (DSM) for Trusses


## Learning Objectives

By the end of this lecture, you will be able to:
- Build a local-to-global transformation for a truss member
- Compute element stiffness in global coordinates: $[k]_g = [T]^T[k'][T]$
- Assemble the global stiffness matrix $[K]$ by scatter-add
- Explain why an unsupported structure yields a singular stiffness matrix
- Apply boundary conditions via partitioning and solve for displacements
- Recover member axial forces from global displacements


## Agenda

**Part 2 (today): Global behavior and the Direct Stiffness Method**
1. Transformation from local to global (direction cosines)
2. Member stiffness relations in the global coordinate system
3. Nodal equilibrium and why assembly works
4. Assemble the global stiffness matrix $[K]$
5. First attempt: no supports (singular $[K]$)
6. Constraints and supports
7. Partitioning into free vs restrained DOFs
8. Solve for global displacements
9. Second attempt: with supports
10. Recover element forces in local coordinates
11. DSM summary: step-by-step procedure
12. Stiffness matrix features + indeterminacy


Big idea:
- A truss is a network of axial springs.
- Each element contributes stiffness to shared DOFs.
- Assembly is adding contributions into the right global rows/columns.


## Part 1 — Local to Global Transformation

### Truss Element in a Structure

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.55;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_TrussGlobal.png" style="width:100%; height:auto;">
    </figure>
  </div>
  <div style="flex:0.45; font-size:0.95em;">
    <ul>
      <li>A truss member is embedded in a <strong>global coordinate system</strong> $(X,Y)$</li>
      <li>The element stiffness was derived in a <strong>local coordinate system</strong> $(x,y)$ aligned with the member</li>
      <li>The member orientation is defined by an angle $\theta$, measured <strong>counterclockwise</strong> from global $X$ to local $x$</li>
      <li>Structural assembly requires <strong>transforming forces and displacements</strong> between local and global coordinates</li>
    </ul>
  </div>
</div>

### Transformation Perspectives

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.95;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_Local2Global.png" style="width:80%; height:auto;">
      <figcaption style="font-size:0.75em; margin-top:0.4em;">
        <!-- Local (left) vs. global (right) descriptions of the same member -->
      </figcaption>
    </figure>
  </div>
</div>

- **Local coordinate system (left):** forces $\boldsymbol{Q}$, displacements $\boldsymbol{u}$
- **Global coordinate system (right):** forces $\boldsymbol{F}$, displacements $\boldsymbol{v}$

### Global → Local Forces (Trigonometry)

<div style="display:flex; gap:1.5rem; align-items:center;">

  <!-- Figures (left, stacked) -->
  <div style="flex:0.45; display:flex; flex-direction:column; gap:0.15rem;">
    <figure style="margin:0; text-align:center;">
      <img src="assets/L2_Local2Global_Member.png" style="width:100%; height:auto;">
    </figure>
    <figure style="margin:0; text-align:center;">
      <img src="assets/L2_Local2Global_Trig.png" style="width:100%; height:auto;">
    </figure>
  </div>

  <!-- Bullets with equations (right) -->
  <div style="flex:0.55; font-size:0.95em;">
    <ul>
      <strong>At node $b$ (start node):</strong>
      <ul>
        <li>$$Q_1 = F_1\cos\theta + F_2\sin\theta$$</li>
        <li>$$Q_2 = -F_1\sin\theta + F_2\cos\theta$$</li>
      </ul>
      <strong>At node $e$ (end node):</strong>
      <ul>
        <li>$$Q_3 = F_3\cos\theta + F_4\sin\theta$$</li>
        <li>$$Q_4 = -F_3\sin\theta + F_4\cos\theta$$</li>
      </ul>
    </ul>
  </div>

</div>


### Global → Local Force Transformation (Matrix Form)

- Local member forces $\boldsymbol{Q}$ are obtained by **rotating** global nodal forces $\boldsymbol{F}$ into the member’s local coordinate system
- Each $2\times2$ block applies a **rotation by $\theta$** at a node
- The transformation changes **direction only**, not force magnitude
- This operation is a **pure coordinate transformation**

$$
\begin{Bmatrix}Q_1\\Q_2\\Q_3\\Q_4\end{Bmatrix}
=
\underbrace{
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
}_{\boldsymbol{T}}
\begin{Bmatrix}F_1\\F_2\\F_3\\F_4\end{Bmatrix}
$$

$$\boldsymbol{Q} = \boldsymbol{T}\boldsymbol{F}$$

### Direction Cosines (Rotation Terms)

- Direction cosines define the **orientation of a truss member** in the global $(X,Y)$ coordinate system
- The angle $\theta$ is measured **counterclockwise** from the global $X$ axis to the local $x$ axis
- Computed directly from the **nodal coordinates** of the element (b = start node, e = end node)

$$
\cos\theta = \frac{X_e - X_b}{\sqrt{(X_e-X_b)^2 + (Y_e-Y_b)^2}},
\qquad
\sin\theta = \frac{Y_e - Y_b}{\sqrt{(X_e-X_b)^2 + (Y_e-Y_b)^2}}
$$

- The denominator is the **member length** $L$
- Once computed, $\cos\theta$ and $\sin\theta$ are **reused throughout the element formulation**


### Global → Local Displacements

- Nodal displacements are transformed using the **same rotation matrix** as forces
- Displacements and forces transform identically because they are defined along the **same directions**
- This is a **pure coordinate rotation**, not a change in deformation

$$
\boldsymbol{u} = \boldsymbol{T}\boldsymbol{v}
$$

- $\boldsymbol{v}$: global displacement vector
- $\boldsymbol{u}$: local displacement vector

### Local → Global Force Transformation

- This is the **reverse of the global → local process**
- Local member forces are **rotated back** into the global $(X,Y)$ directions

**At node $b$ (start node):**
$$
F_1 = Q_1\cos\theta - Q_2\sin\theta,
\qquad
F_2 = Q_1\sin\theta + Q_2\cos\theta
$$

**At node $e$ (end node):**
$$
F_3 = Q_3\cos\theta - Q_4\sin\theta,
\qquad
F_4 = Q_3\sin\theta + Q_4\cos\theta
$$

### Local → Global Force Transformation (Matrix Form)

- Local member forces are mapped to global nodal forces using the **transpose** of the global→local transformation

$$
\begin{Bmatrix}
F_1\\ F_2\\ F_3\\ F_4
\end{Bmatrix}
=
\underbrace{
\begin{bmatrix}
\cos\theta & -\sin\theta & 0 & 0\\
\sin\theta & \cos\theta  & 0 & 0\\
0 & 0 & \cos\theta & -\sin\theta\\
0 & 0 & \sin\theta & \cos\theta
\end{bmatrix}
}_{\boldsymbol{T}^{\mathsf{T}}}
\begin{Bmatrix}
Q_1\\ Q_2\\ Q_3\\ Q_4
\end{Bmatrix}
$$

$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}} \boldsymbol{Q}
$$

**Recall (Global → Local):**
$$
\boldsymbol{T}
=
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
$$


### Properties of the Transformation Matrix

$$\boldsymbol{T}^{-1} = \boldsymbol{T}^T$$

The transformation matrix is **orthogonal**, which greatly simplifies operations involving the stiffness transformations.

### Summary — Local ↔ Global Transformations

**Direction cosines (member orientation):**
$$
\cos\theta = \frac{X_e-X_b}{L}, \qquad \sin\theta = \frac{Y_e-Y_b}{L}
$$

**Global → Local (forces or displacements):**
$$
\boldsymbol{Q} = \boldsymbol{T}\boldsymbol{F},
\qquad
\boldsymbol{u} = \boldsymbol{T}\boldsymbol{v}
$$

**Local → Global (forces or displacements):**
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{Q},
\qquad
\boldsymbol{v} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{u}
$$

$$
\boldsymbol{T}
=
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
$$

- $\boldsymbol{T}$ is a **pure rotation matrix**
- $\boldsymbol{T}^{-1} = \boldsymbol{T}^{\mathsf{T}}$ (orthogonal)


## Part 2 — Member Stiffness in the Global Coordinate System

### Goal

- We have derived the **local stiffness relation** (Lecture 3.1 today):
  $$\boldsymbol{Q} = \boldsymbol{k}\boldsymbol{u}$$
- We also know how to **transform forces and displacements** between local and global systems
- Objective: express the **member stiffness relation entirely in global coordinates**

### Transformation Chain — From Local to Global (Step-by-Step)

**Step 1 — Local force–displacement relation**
$$
\boldsymbol{Q} = \boldsymbol{k}\,\boldsymbol{u}
$$
The element stiffness matrix $\boldsymbol{k}$ relates **local nodal displacements** $\boldsymbol{u}$ to the corresponding **local nodal forces** $\boldsymbol{Q}$.

**Step 2 — Transform local forces to global forces**
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\,\boldsymbol{Q}
$$
Global nodal forces are obtained by rotating the local force vector into the global coordinate system.

Substitute the local stiffness relation from Step 1, $\boldsymbol{Q} = \boldsymbol{k}\,\boldsymbol{u}$, into the force transformation:
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{Q}
\;\;\Longrightarrow\;\;
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\bigl(\boldsymbol{k}\,\boldsymbol{u}\bigr)
$$


**Step 3 — Transform global displacements to local displacements**
$$
\boldsymbol{u} = \boldsymbol{T}\,\boldsymbol{v}
$$

Substitute the displacement transformation into the previous expression:
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{k}\,\boldsymbol{u}
\;\;\Longrightarrow\;\;
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{k}\bigl(\boldsymbol{T}\,\boldsymbol{v}\bigr)
$$


**Step 4 — Rearrange into global stiffness form**
$$
\boldsymbol{F} = (\boldsymbol{T}^{\mathsf{T}}\,\boldsymbol{k}\,\boldsymbol{T})\,\boldsymbol{v}
$$




**Step 5 — Final global stiffness relation**

Define the global element stiffness matrix:
$$
\boldsymbol{K} = \boldsymbol{T}^{\mathsf{T}}\,\boldsymbol{k}\,\boldsymbol{T}
$$

This is the element stiffness relation used directly in global assembly for the Direct Stiffness Method.

textbook notation:
$$
\boxed{\boldsymbol{F} = \boldsymbol{K}\,\boldsymbol{v}}
$$

our notation (interchangeable):
$$
\boxed{\boldsymbol{f} = \boldsymbol{K}\,\boldsymbol{u}}
$$


### Calculating the Global Stiffness Matrix, $\boldsymbol{K}$

The global element stiffness matrix is obtained by **rotating the local axial stiffness**
into the global coordinate system:

<br/>

$$
{\scriptsize
\boldsymbol{K}
=
\begin{bmatrix}
\cos\theta & -\sin\theta & 0 & 0\\
\sin\theta & \cos\theta  & 0 & 0\\
0 & 0 & \cos\theta & -\sin\theta\\
0 & 0 & \sin\theta & \cos\theta
\end{bmatrix}
\cdot
\frac{EA}{L}
\begin{bmatrix}
1 & 0 & -1 & 0\\
0 & 0 & 0 & 0\\
-1 & 0 & 1 & 0\\
0 & 0 & 0 & 0
\end{bmatrix}
\cdot
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
}
$$

<br/>

Carrying out the matrix multiplication yields the **closed-form global stiffness matrix**:

$$
\boldsymbol{K} = \frac{EA}{L}
\begin{bmatrix}
\cos^2\theta & \cos\theta\sin\theta & -\cos^2\theta & -\cos\theta\sin\theta \\
\cos\theta\sin\theta & \sin^2\theta & -\cos\theta\sin\theta & -\sin^2\theta \\
-\cos^2\theta & -\cos\theta\sin\theta & \cos^2\theta & \cos\theta\sin\theta \\
-\cos\theta\sin\theta & -\sin^2\theta & \cos\theta\sin\theta & \sin^2\theta
\end{bmatrix}
$$



### Key Observations — Global Stiffness Matrix, $\boldsymbol{K}$

- The member global stiffness matrix $\boldsymbol{K}$ is **symmetric**, just like the local stiffness matrix  
- $\boldsymbol{K}$ represents the same physical behavior, but expressed in the **global $(X,Y)$ coordinate system**
- Each coefficient $K_{ij}$ is the **force at global DOF $i$** required to produce a **unit displacement at global DOF $j$**, with all other displacements fixed


### Direct Calculation of Global Stiffness Matrix, $\boldsymbol{K}$

- One *could* derive $\boldsymbol{K}$ directly by:
  - Applying **unit global displacements** to a generic inclined truss member
  - Evaluating the **global end forces** required to produce each unit displacement in global coordinates
- The **$j$th column of $\boldsymbol{K}$** gives the global nodal force pattern caused by $v_j=1$

- This approach is **theoretically equivalent** to the transformation-based derivation and provides a clear physical interpretation of $\boldsymbol{K}$.

- However, it is **significantly more labor-intensive**, and is mainly useful as a **verification tool**, rather than for routine analysis.


### Example: First Column of $\boldsymbol{K}$

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.85;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_GlobalDerive1.png" style="width:80%; height:auto;">
      <figcaption style="font-size:0.75em; margin-top:0.4em;">
        <!-- Local (left) vs. global (right) descriptions of the same member -->
      </figcaption>
    </figure>
  </div>
</div>

- Impose $v_1 = 1$, all other global displacements zero
- Project the resulting axial deformation onto the member axis
- Resolve the axial force back into global components

You recover the **first column of $\boldsymbol{K}$**, which should exactly match $\boldsymbol{T}^\mathsf{T}\boldsymbol{k}\boldsymbol{T}$.

## Part 3 — Assembling the Global Structure Stiffness Matrix (Manual Method)

### Example Structure

We now move from **element-level** to a **complete truss structure** behavior.

- Structure composed of **3 axial truss elements**
- **4 nodes**, each with $(X,Y)$ translational DOFs, node 1 at the top
- Total system size: **8 global degrees of freedom**
  - node 1: DOF (1,2)
  - node 2: DOF (3,4)
  - node 3: DOF (5,6)
  - node 4: DOF (7,8)

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.5;">
    <img src="assets/L2_Structure1.png" style="width:80%;">
  </div>
  <div style="flex:0.5;">
    <img src="assets/L2_Structure2.png" style="width:80%;">
  </div>
</div>

### Element-Level View of the Structure

<div style="text-align:center;">
  <img src="assets/L2_Structure3.png" style="width:80%;">
</div>

- Nodes **2, 3, and 4** are **pinned** (no displacement)
- **Node 1** is free to move
- All three elements are connected at **node 1**
- A displacement at node 1 induces forces in **all connected members**

### Element Forces and Notation

Local forces are defined **per element**:

- Superscript $(e)$ → element number
- Subscript $(i)$ → local DOF index

Examples:
- $F^{(1)}_3$ → force at local DOF 3 in **element 1**
- $F^{(2)}_1$ → force at local DOF 1 in **element 2**



### Equilibrium Equations at Node 1

Because all elements share **node 1**, equilibrium at this node couples the response of all members.

$P$ = external applied loaded

$F$ = internal forces

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.55;">
    <img src="assets/L2_Structure4.png" style="width:80%;">
  </div>
  <div style="flex:0.45; font-size:0.95em;">
    <p><strong>Force equilibrium at node 1:</strong></p>
    $$P_1 = F_3^{(1)} + F_1^{(2)} + F_3^{(3)}$$
    $$P_2 = F_4^{(1)} + F_2^{(2)} + F_4^{(3)}$$
  </div>
</div>

### Compatability Equations

<div style="text-align:center;">
  <img src="assets/L2_Structure3.png" style="width:70%;">
</div>

$d_1$ and $d_2$ are global displacements in X and Y at node 1

$$
\begin{aligned}
\text{Member }(1): \quad
& v^{(1)}_1 = v^{(1)}_2 = 0,
&& v^{(1)}_3 = d_1,
&& v^{(1)}_4 = d_2 \\[6pt]
\text{Member }(2): \quad
& v^{(2)}_1 = d_1,
&& v^{(2)}_2 = d_2,
&& v^{(2)}_3 = v^{(2)}_4 = 0 \\[6pt]
\text{Member }(3): \quad
& v^{(3)}_1 = v^{(3)}_2 = 0,
&& v^{(3)}_3 = d_1,
&& v^{(3)}_4 = d_2
\end{aligned}
$$


### Member 1 — Force–Displacement Relations

Compatibility (Member 1): $v^{(1)}_1 = v^{(1)}_2 = 0,\; v^{(1)}_3 = d_1,\; v^{(1)}_4 = d_2$

Element-level global stiffness expression:

$$
{\small
\begin{Bmatrix}
F^{(1)}_1\\
F^{(1)}_2\\
F^{(1)}_3\\
F^{(1)}_4
\end{Bmatrix}
=
\begin{bmatrix}
K^{(1)}_{11} & K^{(1)}_{12} & K^{(1)}_{13} & K^{(1)}_{14}\\
K^{(1)}_{21} & K^{(1)}_{22} & K^{(1)}_{23} & K^{(1)}_{24}\\
K^{(1)}_{31} & K^{(1)}_{32} & \boxed{K^{(1)}_{33}} & \boxed{K^{(1)}_{34}}\\
K^{(1)}_{41} & K^{(1)}_{42} & \boxed{K^{(1)}_{43}} & \boxed{K^{(1)}_{44}}
\end{bmatrix}
\begin{Bmatrix}
v^{(1)}_1 = 0\\
v^{(1)}_2 = 0\\
v^{(1)}_3 = d_1\\
v^{(1)}_4 = d_2
\end{Bmatrix}
}
$$

The forces acting at **end node of member 1** (Global node 1) are:

$$
F^{(1)}_3
=
K^{(1)}_{33} d_1
+
K^{(1)}_{34} d_2
$$

$$
F^{(1)}_4
=
K^{(1)}_{43} d_1
+
K^{(1)}_{44} d_2
$$


### Member 2 — Force–Displacement Relations

Compatibility (Member 2): $v^{(2)}_1 = d_1,\; v^{(2)}_2 = d_2,\; v^{(2)}_3 = v^{(2)}_4 = 0$

Element-level global stiffness expression:

$$
{\small
\begin{Bmatrix}
F^{(2)}_1\\
F^{(2)}_2\\
F^{(2)}_3\\
F^{(2)}_4
\end{Bmatrix}
=
\begin{bmatrix}
\boxed{K^{(2)}_{11}} & \boxed{K^{(2)}_{12}} & K^{(2)}_{13} & K^{(2)}_{14}\\
\boxed{K^{(2)}_{21}} & \boxed{K^{(2)}_{22}} & K^{(2)}_{23} & K^{(2)}_{24}\\
K^{(2)}_{31} & K^{(2)}_{32} & K^{(2)}_{33} & K^{(2)}_{34}\\
K^{(2)}_{41} & K^{(2)}_{42} & K^{(2)}_{43} & K^{(2)}_{44}
\end{bmatrix}
\begin{Bmatrix}
v^{(2)}_1 = d_1\\
v^{(2)}_2 = d_2\\
v^{(2)}_3 = 0\\
v^{(2)}_4 = 0
\end{Bmatrix}
}
$$

The forces acting at **start node of member 2** (Global node 1) are:

$$
F^{(2)}_1
=
K^{(2)}_{11} d_1
+
K^{(2)}_{12} d_2
$$

$$
F^{(2)}_2
=
K^{(2)}_{21} d_1
+
K^{(2)}_{22} d_2
$$


### Member 3 — Force–Displacement Relations

Compatibility (Member 3): $v^{(3)}_1 = v^{(3)}_2 = 0,\; v^{(3)}_3 = d_1,\; v^{(3)}_4 = d_2$

Element-level global stiffness expression:

$$
{\small
\begin{Bmatrix}
F^{(3)}_1\\
F^{(3)}_2\\
F^{(3)}_3\\
F^{(3)}_4
\end{Bmatrix}
=
\begin{bmatrix}
K^{(3)}_{11} & K^{(3)}_{12} & K^{(3)}_{13} & K^{(3)}_{14}\\
K^{(3)}_{21} & K^{(3)}_{22} & K^{(3)}_{23} & K^{(3)}_{24}\\
K^{(3)}_{31} & K^{(3)}_{32} & \boxed{K^{(3)}_{33}} & \boxed{K^{(3)}_{34}}\\
K^{(3)}_{41} & K^{(3)}_{42} & \boxed{K^{(3)}_{43}} & \boxed{K^{(3)}_{44}}
\end{bmatrix}
\begin{Bmatrix}
v^{(3)}_1 = 0\\
v^{(3)}_2 = 0\\
v^{(3)}_3 = d_1\\
v^{(3)}_4 = d_2
\end{Bmatrix}
}
$$

The forces acting at **end node of member 3** (Global node 1) are:

$$
F^{(3)}_3
=
K^{(3)}_{33} d_1
+
K^{(3)}_{34} d_2
$$

$$
F^{(3)}_4
=
K^{(3)}_{43} d_1
+
K^{(3)}_{44} d_2
$$

### Global Stiffness Matrix for Free DOFs

$$
\begin{aligned}
P_1 &= F_3^{(1)} + F_1^{(2)} + F_3^{(3)} \\
P_2 &= F_4^{(1)} + F_2^{(2)} + F_4^{(3)}
\end{aligned}
$$

Substituting element-level force expressions into the node 1 force equilibrium gives:
$$
\begin{aligned}
P_1 &=
\bigl(K_{33}^{(1)} + K_{11}^{(2)} + K_{33}^{(3)}\bigr) d_1
+
\bigl(K_{34}^{(1)} + K_{12}^{(2)} + K_{34}^{(3)}\bigr) d_2 \\[6pt]
P_2 &=
\bigl(K_{43}^{(1)} + K_{21}^{(2)} + K_{43}^{(3)}\bigr) d_1
+
\bigl(K_{44}^{(1)} + K_{22}^{(2)} + K_{44}^{(3)}\bigr) d_2
\end{aligned}
$$

These equations can be written compactly as:
$$
\begin{Bmatrix}
P_1 \\ P_2
\end{Bmatrix}
=
\underbrace{
\begin{bmatrix}
K_{33}^{(1)} + K_{11}^{(2)} + K_{33}^{(3)} &
K_{34}^{(1)} + K_{12}^{(2)} + K_{34}^{(3)} \\[4pt]
K_{43}^{(1)} + K_{21}^{(2)} + K_{43}^{(3)} &
K_{44}^{(1)} + K_{22}^{(2)} + K_{44}^{(3)}
\end{bmatrix}
}_{\boldsymbol{K}_{\text{s}}}
\begin{Bmatrix}
d_1 \\ d_2
\end{Bmatrix}
$$

$$
\boxed{\;\boldsymbol{P} = \boldsymbol{K}_{\text{s}}\,\boldsymbol{d}\;}
$$

### Physical Interpretation of the Structure Stiffness Matrix

- The **structure stiffness matrix** $\boldsymbol{K}_s$ has the same physical meaning as an element stiffness matrix, but at the **structure (joint) level**
- A stiffness coefficient $K_{s,ij}$ represents:
  - The **joint force** at DOF $i$
  - Required to cause a **unit displacement** at DOF $j$
  - While **all other joint displacements are zero**
- Equivalently:
  - Each **column** of $\boldsymbol{K}_s$ corresponds to a unit displacement pattern
  - The column entries are the **resulting joint forces** needed to enforce that displacement


### Example — First Column of $\boldsymbol{K}_s$

- To obtain the **first column** of $\boldsymbol{K}_s$:
  - Impose a **unit displacement** at the first free DOF:
    $$
    d_1 = 1, \qquad d_2 = 0
    $$
  - All other joint displacements are held fixed
- The resulting joint forces (DOF 1, 2) define the **first column** of $\boldsymbol{K}_s$


<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.5;">
    <img src="assets/L2_GlobalManual1.png" style="width:90%;">
  </div>
  <div style="flex:0.5;">
    <img src="assets/L2_GlobalManual2.png" style="width:90%;">
  </div>
</div>


## Part 4 — Assembling the Global Structure Stiffness Matrix

**Direct Stiffness Method**

### Why Not the Manual (Equilibrium + Compatibility) Approach?

- Becomes **tedious** very quickly
- Requires manually tracking:
  - Free vs constrained DOFs
  - Compatibility relationships
- Hard to implement **programmatically**
- Does **not scale** to larger or changing structures

**Key idea:** We want a method that is systematic, scalable, and algorithmic.

### Reminder — Global Degrees of Freedom

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.6;">
    <img src="assets/L2_Structure2_DOFs.png" style="width:100%;">
  </div>
  <div style="flex:0.4; font-size:0.95em;">
    <ul>
      <li>Each node contributes two DOFs: $(X,Y)$</li>
      <li>4 nodes → <strong>8 global DOFs</strong></li>
      <li>Consistent numbering is essential for assembly</li>
    </ul>
  </div>
</div>

### Map Element DOFs to Global DOFs

<img src="assets/L2_Mapping1.png" style="width:100%; margin-bottom:0.75rem;">

Each element has **4 local DOFs** $(1,2,3,4)$, which must be mapped to the correct **global DOFs** before assembly.

For this structure:

- **Element 1:** local $(1,2,3,4) \;\rightarrow\;$ global $(3,4,1,2)$  
- **Element 2:** local $(1,2,3,4) \;\rightarrow\;$ global $(1,2,5,6)$  
- **Element 3:** local $(1,2,3,4) \;\rightarrow\;$ global $(7,8,1,2)$  

This mapping determines **which rows and columns** of the global stiffness matrix each element stiffness contributes to.


### Method 1: Assemble a Structure Matrix for the Free DOFs

<img src="assets/L2_Mapping2.png" style="width:80%; margin-bottom:0.75rem;">

In this example, only **global DOFs 1 and 2** are free; all other DOFs are constrained.

- Contributions from **each element** that map to DOFs 1 and 2 are **summed**
- This produces a reduced **structure stiffness matrix** for the free DOFs
- The result matches the stiffness matrix obtained earlier using the **manual equilibrium and compatibility** approach

For example, the $(1,1)$ entry of the structure stiffness matrix is (matches what we did before):
$$
K_{s,11} = K_{33}^{(1)} + K_{11}^{(2)} + K_{33}^{(3)}
$$

### Method 2: Assemble the Full Global Stiffness Matrix

When assembling the **global structure stiffness matrix**, do **not** worry yet about which DOFs are constrained or free.  

- **Assemble all DOFs first** into a single global matrix  
- **Apply boundary conditions later** (supports, prescribed displacements)  
- This separation is what makes the method **general, systematic, and scalable**

For the structure shown, there are **8 global DOFs**, so **$8\times 8$** stiffness matrix:

$$
{\small
\boldsymbol{K}=
\begin{bmatrix}
K_{11} & K_{12} & K_{13} & K_{14} & K_{15} & K_{16} & K_{17} & K_{18} \\
K_{21} & K_{22} & K_{23} & K_{24} & K_{25} & K_{26} & K_{27} & K_{28} \\
K_{31} & K_{32} & K_{33} & K_{34} & K_{35} & K_{36} & K_{37} & K_{38} \\
K_{41} & K_{42} & K_{43} & K_{44} & K_{45} & K_{46} & K_{47} & K_{48} \\
K_{51} & K_{52} & K_{53} & K_{54} & K_{55} & K_{56} & K_{57} & K_{58} \\
K_{61} & K_{62} & K_{63} & K_{64} & K_{65} & K_{66} & K_{67} & K_{68} \\
K_{71} & K_{72} & K_{73} & K_{74} & K_{75} & K_{76} & K_{77} & K_{78} \\
K_{81} & K_{82} & K_{83} & K_{84} & K_{85} & K_{86} & K_{87} & K_{88}
\end{bmatrix}
}
$$

Each entry $K_{ij}$ represents the **combined stiffness contribution** from all elements that connect global DOF $j$ to global DOF $i$.


### Element 1 — Contribution to Global Stiffness Matrix

**Local → Global DOF mapping (Element 1):**
$$
(1,2,3,4) \;\rightarrow\; (3,4,1,2)
$$

<img src="assets/L2_Mapping_ele1.png" style="width:60%; margin-bottom:0.75rem;">




### Element 2 — Contribution to Global Stiffness Matrix

**Local → Global DOF mapping (Element 2):**
$$
(1,2,3,4) \;\rightarrow\; (1,2,5,6)
$$

<img src="assets/L2_Mapping_ele2.png" style="width:60%; margin-bottom:0.75rem;">


### Element 3 — Contribution to Global Stiffness Matrix

**Local → Global DOF mapping (Element 3):**
$$
(1,2,3,4) \;\rightarrow\; (7,8,1,2)
$$

<img src="assets/L2_Mapping_ele3.png" style="width:60%; margin-bottom:0.75rem;">


### Final Global Structure Stiffness Matrix

- Each $K_{ij}$ may include **multiple element contributions**
- The matrix is **symmetric**
- Diagonal terms, $K_{ii}$, must be non-zero and positive for stability

$$
\boldsymbol{K}=
\begin{bmatrix}
K_{11} & K_{12} & K_{13} & K_{14} & K_{15} & K_{16} & K_{17} & K_{18} \\
K_{21} & K_{22} & K_{23} & K_{24} & K_{25} & K_{26} & K_{27} & K_{28} \\
K_{31} & K_{32} & K_{33} & K_{34} & 0 & 0 & 0 & 0 \\
K_{41} & K_{42} & K_{43} & K_{44} & 0 & 0 & 0 & 0 \\
K_{51} & K_{52} & 0 & 0 & K_{55} & K_{56} & 0 & 0 \\
K_{61} & K_{62} & 0 & 0 & K_{65} & K_{66} & 0 & 0 \\
K_{71} & K_{72} & 0 & 0 & 0 & 0 & K_{77} & K_{78} \\
K_{81} & K_{82} & 0 & 0 & 0 & 0 & K_{87} & K_{88}
\end{bmatrix}
$$

### Interpreting Zeros in the Stiffness Matrix

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.6;">
    <img src="assets/L2_Structure2_DOFs.png" style="width:100%;">
  </div>
  <div style="flex:0.4; font-size:0.95em;">
    <ul>
      <li>Zero entry → no stiffness coupling</li>
      <li>Example: $K_{37} = 0$</li>
      <li>DOF 3 and DOF 7 share no connecting element</li>
    </ul>
  </div>
</div>

## Part 5 — Constraints and Supports

### Try Inverting

You might think: *we have* $\boldsymbol{K}$, *so we can just solve*
$$
\boldsymbol{K}\,\boldsymbol{u}=\boldsymbol{f}
$$
and we’re done.

But if you try to compute $\boldsymbol{u}=\boldsymbol{K}^{-1}\boldsymbol{f}$ **right now**, you’ll get an error:

- $\boldsymbol{K}$ is **singular** (not invertible)
- the structure still has **rigid-body motion** because we haven’t applied supports / boundary conditions yet


### Why do we get a singular stiffness matrix?

When the global stiffness matrix $\mathbf{K}$ is assembled **without supports**, the structure is free to undergo **rigid-body motion**. These motions do not stretch or compress members and therefore produce **no internal strain energy**.

Mathematically, an unconstrained structure admits non-zero displacement vectors $\mathbf{u} \neq \mathbf{0}$ such that
$$
\mathbf{K}\mathbf{u} = \mathbf{0}
$$
These displacement patterns are **rigid-body (zero-energy) modes**, meaning $\mathbf{K}$ has a **non-trivial nullspace** and is therefore **singular**.

**Bottom line:** without supports, $\mathbf{K}$ cannot uniquely map loads to displacements.


### What does this mean in practice?

Consider the assembled $8 \times 8$ global stiffness matrix $\mathbf{K}$. At this stage, it contains **no information about how the structure is attached to the ground**.

As a result, the structure can move without inducing member deformation:

- **Rigid translation in $X$**
- **Rigid translation in $Y$**
- **Rigid-body rotation** (geometry-dependent)

These motions are **rigid-body modes**, not structural deformations.

Because rigid-body modes produce zero internal force, $\mathbf{K}$ cannot uniquely map applied loads to displacements. **Supports (boundary conditions)** remove these modes and render $\mathbf{K}$ invertible.


### Boundary conditions: how supports enter the model

Supports and constraints are enforced by specifying **known displacements** (a.k.a. prescribed DOFs).

**Common idealized supports (2D truss):**
- **Pin support**: Prescribes both translations: $u_{ix}=0$ and $u_{iy}=0$
- **Roller support**: Prescribes one translation and allows the other:

**Prescribed displacement (settlement / actuation):**
- You can prescribe a non-zero value, e.g., $u_{iy} = -2\text{ mm}$ at a support.

Why this fixes singularity:
- Constraints remove rigid-body modes by preventing global translations/rotations.
- The remaining free DOFs correspond to true structural deformation.


### What we do next (workflow)

We do **not** modify the element --> global matrix formulation to handle supports.

Instead, the Direct Stiffness Method proceeds like this:
1. Assemble the **full** global matrix $\boldsymbol{K}$ and global load vector $\boldsymbol{F}$
2. Identify which DOFs are **free** vs **restrained**
3. Enforce boundary conditions by **partitioning** (or equivalently, by eliminating restrained DOFs)
4. Solve only for the unknown (free) displacements
5. Recover reactions at restrained DOFs

So supports are handled cleanly at the **system equation** level — via partitioning (next section).

## Part 5 — Partitioning the Matrix (Free vs Restrained)

### Core idea

Separate DOFs into:
- **Free DOFs** $f$: unknown displacements (to solve for)
- **Restrained DOFs** $r$: prescribed displacements (known from supports/constraints)

Reorder the global displacement vector so all free DOFs come first:
$$
\boldsymbol{u}=
\begin{bmatrix}
\boldsymbol{u}_f\\
\boldsymbol{u}_r
\end{bmatrix},
\qquad
\boldsymbol{F}=
\begin{bmatrix}
\boldsymbol{F}_f\\
\boldsymbol{F}_r
\end{bmatrix}
$$

Then the global system becomes a block system:
$$
\begin{bmatrix}
\boldsymbol{K}_{ff} & \boldsymbol{K}_{fr}\\
\boldsymbol{K}_{rf} & \boldsymbol{K}_{rr}
\end{bmatrix}
\begin{bmatrix}
\boldsymbol{u}_f\\
\boldsymbol{u}_r
\end{bmatrix}
=
\begin{bmatrix}
\boldsymbol{F}_f\\
\boldsymbol{F}_r
\end{bmatrix}
$$

Interpretation:
- $\boldsymbol{K}_{ff}$ couples free DOFs to free DOFs
- $\boldsymbol{K}_{fr}$ couples restrained DOFs into the free equations
- $\boldsymbol{K}_{rf}$ and $\boldsymbol{K}_{rr}$ are used to recover reactions

### How do we identify free vs restrained DOFs?

For each node in a 2D truss, we typically have two DOFs:
- $u_x$ (global $X$ translation)
- $u_y$ (global $Y$ translation)

A DOF is **restrained** if its displacement is prescribed:
- Pin support at node $i$: restrains both $u_{ix}$ and $u_{iy}$
- Roller at node $i$ (vertical): restrains $u_{iy}$ only
- Any imposed displacement: restrains that DOF to a specified value

A DOF is **free** if its displacement is unknown and must be solved from equilibrium.

In implementation, you typically store:
- a list of free DOF indices, e.g. `free = [ ... ]`
- a list of restrained DOF indices, e.g. `rest = [ ... ]`

Then you form submatrices by indexing into $\boldsymbol{K}$.

### What the submatrices mean (physical interpretation)

Start from the full equilibrium statement:
$$\boldsymbol{K}\boldsymbol{u} = \boldsymbol{F}$$

After partitioning, the **top block row** is:
$$\boldsymbol{K}_{ff}\boldsymbol{u}_f + \boldsymbol{K}_{fr}\boldsymbol{u}_r = \boldsymbol{F}_f$$

This is the equation we solve for unknown displacements $\boldsymbol{u}_f$.

The **bottom block row** is:
$$\boldsymbol{K}_{rf}\boldsymbol{u}_f + \boldsymbol{K}_{rr}\boldsymbol{u}_r = \boldsymbol{F}_r$$

This equation is used to compute **support reactions** once $\boldsymbol{u}_f$ is known.

Important notes:
- $\boldsymbol{F}_r$ are not usually prescribed loads — they are typically **unknown reactions**
- $\boldsymbol{u}_r$ are known (often zero), so they act like a known input

### Non-zero prescribed displacements (settlement / actuation)

Partitioning also handles cases where restrained displacements are not zero.

Example:
- A support settles by $\delta$: $u_{iy} = -\delta$ is prescribed

The free-DOF equation is still:
$$\boldsymbol{K}_{ff}\boldsymbol{u}_f + \boldsymbol{K}_{fr}\boldsymbol{u}_r = \boldsymbol{F}_f$$

Rearranged into a solve-ready form:
$$\boldsymbol{K}_{ff}\boldsymbol{u}_f = \boldsymbol{F}_f - \boldsymbol{K}_{fr}\boldsymbol{u}_r$$

So, prescribed support motion enters as an *effective load term* on the right-hand side.

## Part 6 — Solving for Global Displacements and Forces

### Solve the partitioned system

Start from the partitioned equilibrium equations:
$$
\boldsymbol{K}_{ff}\boldsymbol{u}_f + \boldsymbol{K}_{fr}\boldsymbol{u}_r = \boldsymbol{F}_f
$$
$$
\boldsymbol{K}_{rf}\boldsymbol{u}_f + \boldsymbol{K}_{rr}\boldsymbol{u}_r = \boldsymbol{F}_r
$$

Typically, $\boldsymbol{u}_r$ is known (often $\boldsymbol{0}$). Solve for the unknown displacements:
$$
\boxed{\;\boldsymbol{u}_f = \boldsymbol{K}_{ff}^{-1}\Bigl(\boldsymbol{F}_f - \boldsymbol{K}_{fr}\boldsymbol{u}_r\Bigr)\;}
$$

Then compute reactions at the supports (restrained DOFs):
$$
\boxed{\;\boldsymbol{F}_r = \boldsymbol{K}_{rf}\boldsymbol{u}_f + \boldsymbol{K}_{rr}\boldsymbol{u}_r\;}
$$

Interpretation:
- $\boldsymbol{u}_f$ gives the structural deformation
- $\boldsymbol{F}_r$ gives the support reactions required to enforce the boundary conditions

### Algorithmic view (what you do in code)

Given:
- full global stiffness matrix $\boldsymbol{K}$
- full global load vector $\boldsymbol{F}$
- sets of indices `free` and `rest`
- prescribed displacements $\boldsymbol{u}_r$ (often zeros)

Steps:
1. Extract submatrices:
   - $\boldsymbol{K}_{ff} = \boldsymbol{K}[f,f]$
   - $\boldsymbol{K}_{fr} = \boldsymbol{K}[f,r]$
   - $\boldsymbol{K}_{rf} = \boldsymbol{K}[r,f]$
   - $\boldsymbol{K}_{rr} = \boldsymbol{K}[r,r]$
2. Extract subvectors:
   - $\boldsymbol{F}_f = \boldsymbol{F}[f]$
   - $\boldsymbol{F}_r = \boldsymbol{F}[r]$ (usually unknown reactions; can initialize as zeros)
3. Solve:
   $$\boldsymbol{u}_f = \boldsymbol{K}_{ff}^{-1}(\boldsymbol{F}_f - \boldsymbol{K}_{fr}\boldsymbol{u}_r)$$
4. Recover reactions:
   $$\boldsymbol{F}_r = \boldsymbol{K}_{rf}\boldsymbol{u}_f + \boldsymbol{K}_{rr}\boldsymbol{u}_r$$

Practical note:
- In numerical computing, you rarely compute $\boldsymbol{K}_{ff}^{-1}$ explicitly.
- You solve $\boldsymbol{K}_{ff}\boldsymbol{u}_f = \boldsymbol{b}$ using a linear solver.

### Sanity checks after solving

**1) Dimensions and indexing**
- $\boldsymbol{K}_{ff}$ should be $n_f\times n_f$
- $\boldsymbol{u}_f$ should be length $n_f$

**2) Stability**
- If the structure is properly constrained, $\boldsymbol{K}_{ff}$ should be non-singular
- If it is still singular, you likely have:
  - insufficient supports (rigid-body mode remains), or
  - a disconnected node / element connectivity error

**3) Equilibrium**
- Reconstruct the full displacement vector $\boldsymbol{u}$ by inserting $\boldsymbol{u}_f$ and $\boldsymbol{u}_r$
- Compute $\boldsymbol{K}\boldsymbol{u}$ and compare to $\boldsymbol{F}$
- Differences should be near numerical tolerance (floating point error)

**4) Reactions**
- Check reaction directions and magnitudes against intuition
- Sum of reactions should balance applied loads (global equilibrium)


## Part 7 — Recover Element Forces (Back to Local)

> **Draft note (delete later):**
> now go back to local, for element forces etc. Section 3.8 Kassimalu explains, in the step by step guide

For each member:
1. Extract element global displacement vector $\{u\}_e$
2. Transform to local: $\{u'\} = [T]\{u\}_e$
3. Compute local end forces: $\{f'\} = [k']\{u'\}$
4. Axial force: $N = \frac{EA}{L}(u_2' - u_1')$


## Part 8 — DSM Summary (Explicit Steps)

> **Draft note (delete later):**
> the direct stiffness method summary (what we just did), steps to solve, explicit steps

1. Define geometry (nodes, members)
2. Number DOFs
3. For each member:
   - compute $L,\; \theta,\; c,\; s$
   - build $[T]$
   - compute $[k]_g = [T]^T[k'][T]$
4. Assemble $[K]$
5. Apply boundary conditions (partition into $f,r$)
6. Solve for $u_f$
7. Recover reactions $F_r$
8. Recover member forces/stresses in local coordinates


## Part 9 — Features of the Stiffness Matrix + Indeterminacy

> **Draft note (delete later):**
> some features of the stiffness matrix + indeterminacy (mcGuire 3.3, 3.4)

Typical properties (for stable, properly constrained trusses):
- Symmetric
- Sparse
- Positive definite on free DOFs

Indeterminacy (conceptual):
- More members than needed for determinacy: stiffness method still works
- The matrix system enforces compatibility and equilibrium automatically


## Part 10 — Worked Example: Second Attempt (With Supports)

> **Draft note (delete later):**
> now solve McGuire 3.2, same structure as 3.1 but now with supports

Now apply supports and solve for:
- global displacements $\{u\}$
- reactions at restrained DOFs


In [ ]:
# (Optional) DSM code scaffold stub
# Placeholder for the hands-on notebook (or Lecture 3 lab section).

import numpy as np

# TODO: define nodes, connectivity, E, A
nodes = None          # e.g., np.array([[x1,y1],[x2,y2],...])
members = None        # e.g., list of (n1, n2)
E = None
A = None

# TODO: DOF numbering, element k_g, assembly, partitioning, solve, recovery
K = None
F = None
u = None

K, F, u


## Wrap-Up

Today you built the DSM pipeline for trusses:
- local bar stiffness $\to$ transformation $\to$ global element stiffness
- assembly $\to$ constraints $\to$ solve $\to$ member force recovery

Next: implement DSM in Python for a worked truss example and discuss efficiency (sparsity/bandedness).
